<a href="https://colab.research.google.com/github/Kate777-hue/ADpredict/blob/main/6300_ad_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DATA 6300 Final Project Overview


### Exploratory Data Analysis (EDA)
- Analyze the relationships between features and the target variable (` Diagnosis`) to uncover meaningful patterns and trends.

### Data Preprocessing
- Remove irrelevant features, including PatientID and  DoctorInCharge.


### Model Development
- Train Baseline Models:
  - Logistic Regression
  - Support Vector Machine
  - Random Forest
  - XGBoost
- Train Ensemble Models
  - Voting
  - Stacking
- Analyze Feature Importance
- Hyperparameter Tuning


### Model Evaluation
- Evaluate models using Accuracy, Precision, Recall, F1 Score.
- Compare performance across models to identify the best balance between parameters and generalization.

#Import Relevant Libraries

In [ ]:
!pip install -U ydata-profiling
from ydata_profiling import ProfileReport
from google.colab import files
import numpy as np
import pandas as pd
import statsmodels.api as sm
from matplotlib.pyplot import subplots
import sklearn.model_selection as skm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import RandomizedSearchCV , GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc
import random
import tensorflow as tf
import torch


SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)
torch.manual_seed(SEED)

#Import Data

In [ ]:
# Import the data using google colab
uploaded = files.upload()

In [ ]:
# Load dataset
df = pd.read_csv('alzheimers_disease_data.csv')
df.info()

#Exploratory Data Analysis

In [ ]:
ProfileReport(df)

In [ ]:
# Drop the specified columns as irrelevant variables
df.drop(['PatientID', 'DoctorInCharge'], axis=1, inplace=True)

# Display the first few rows to verify the columns are dropped
print(df.head())

##Define Features and Target

In [ ]:
X = df.drop('Diagnosis', axis=1)
y = df['Diagnosis']

##Separate Numercial and categorical variables

In [ ]:
# Define a threshold: if a numeric column has fewer unique values than this, consider it categorical.
unique_threshold = 10

numerical_vars = []
categorical_vars = []

for col in X.columns:
    # Check if the column's datatype is numeric
    if pd.api.types.is_numeric_dtype(X[col]):
        unique_count = X[col].nunique()
        if unique_count < unique_threshold:
            categorical_vars.append(col)
        else:
            numerical_vars.append(col)
    else:
        # If not numeric, we assume it's categorical
        categorical_vars.append(col)

# Print the lists and their counts
print("Numerical Variables:", numerical_vars)
print("Count of Numerical Variables:", len(numerical_vars))
print("\nCategorical Variables:", categorical_vars)
print("Count of Categorical Variables:", len(categorical_vars))

In [ ]:
#split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
# Create a copy of the training and testing data
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the numerical columns of the training set and transform them
#scaler.fit(X_train[numerical_vars])
#print("Mean:", scaler.mean_)
#print("Scale:", scaler.scale_)
#joblib.dump(scaler, "scaler_model.pkl")
#X_train_scaled[numerical_vars] = scaler.transform(X_train[numerical_vars])
X_train_scaled[numerical_vars] = scaler.fit_transform(X_train[numerical_vars])


# Transform the numerical columns of the testing set using the fitted scaler
X_test_scaled[numerical_vars] = scaler.fit_transform(X_test[numerical_vars])

# Optionally, print a few rows to verify
print("Scaled Training Data (first 5 rows):")
print(X_train_scaled.head())

In [ ]:
# Combine scaled features with the target for the training set
train_data = X_train_scaled.copy()
train_data['Diagnosis'] = y_train

# Combine scaled features with the target for the test set
test_data = X_test_scaled.copy()
test_data['Diagnosis'] = y_test

# Save the training and testing datasets as CSV files
train_data.to_csv('train_dataset.csv', index=False)
test_data.to_csv('test_dataset.csv', index=False)

print("Training and testing datasets have been saved as 'train_dataset.csv' and 'test_dataset.csv'.")


In [ ]:
  #load saved files

files.download('train_dataset.csv')
files.download('test_dataset.csv')

#Modelling and Model Tuning

In [ ]:
# #load train data
# train_data = pd.read_csv('train_dataset.csv')
# train_data.info()

# #load test data
# test_data = pd.read_csv('test_dataset.csv')
# test_data.info()

##Train Baseline Models


In [ ]:
# --- Identify features and target ---

X_train = train_data.drop('Diagnosis', axis=1)
y_train = train_data['Diagnosis']

X_test = test_data.drop('Diagnosis', axis=1)
y_test = test_data['Diagnosis']

In [ ]:
# --- Baseline Model 1: Logistic Regression ---
print("\n--- Logistic Regression ---")
lr = LogisticRegression(random_state=SEED, max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Classification Report:")
print(classification_report(y_test, y_pred_lr))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_lr))

In [ ]:
# --- Baseline Model 2: Support Vector Machines ---
print("\n--- Support Vector Machine ---")
svm = SVC(kernel='rbf', probability=True, random_state=SEED)
svm.fit(X_train, y_train)

y_pred_svm = svm.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Classification Report:")
print(classification_report(y_test, y_pred_svm))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_svm))

In [ ]:
# --- Baseline Model 3: Random Forest ---
print("\n--- Random Forest ---")
rf = RandomForestClassifier(random_state=SEED)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:")
print(classification_report(y_test, y_pred_rf))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

In [ ]:
# --- Baseline Model 4: XGBoost ---
print("\n--- XGBoost ---")
xgb_model = XGBClassifier(random_state=SEED, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# --- Make predictions ---
y_pred_xgb = xgb_model.predict(X_test)

# --- Evaluate the model ---
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("Classification Report:")
print(classification_report(y_test, y_pred_xgb))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_xgb))

##Ensemble Model 1: Voting

In [ ]:
# Create Voting Classifier 1
# with Logistic Regression, Random Forest and XGBOOST

voting_ensemble_1 = VotingClassifier(
    estimators=[('lr', lr), ('rf', rf), ('xgb', xgb_model)],
    voting='hard'  # or use 'soft' if models support predict_proba()
)
voting_ensemble_1.fit(X_train, y_train)
y_pred_voting_1 = voting_ensemble_1.predict(X_test)

print("Voting Ensemble 1 Accuracy:", accuracy_score(y_test, y_pred_voting_1))
print("Classification Report for Voting Ensemble 1:")
print(classification_report(y_test, y_pred_voting_1))
print("Confusion Matrix for Voting Ensemble 1:")
print(confusion_matrix(y_test, y_pred_voting_1))


# Create Voting Classifier 2
# with Logistic Regression, SVM, Random Forest and XGBOOST

voting_ensemble_2 = VotingClassifier(
    estimators=[('lr', lr), ('svm', svm), ('rf', rf),('xgb', xgb_model)],
    voting='hard'  # or use 'soft' if models support predict_proba()
)
voting_ensemble_2.fit(X_train, y_train)
y_pred_voting_2 = voting_ensemble_2.predict(X_test)

print("Voting Ensemble 2 Accuracy:", accuracy_score(y_test, y_pred_voting_2))
print("Classification Report for Voting Ensemble 2:")
print(classification_report(y_test, y_pred_voting_2))
print("Confusion Matrix for Voting Ensemble 2:")
print(confusion_matrix(y_test, y_pred_voting_2))


# Create Voting Classifier 3
# with SVM, Random Forest and XGBOOST

voting_ensemble_3 = VotingClassifier(
    estimators=[('svm', svm), ('rf', rf),('xgb', xgb_model)],
    voting='hard'  # or use 'soft' if models support predict_proba()
)
voting_ensemble_3.fit(X_train, y_train)
y_pred_voting_3 = voting_ensemble_3.predict(X_test)

print("Voting Ensemble 3 Accuracy:", accuracy_score(y_test, y_pred_voting_3))
print("Classification Report for Voting Ensemble 3:")
print(classification_report(y_test, y_pred_voting_3))
print("Confusion Matrix for Voting Ensemble 3:")
print(confusion_matrix(y_test, y_pred_voting_3))



##Ensemble Model 2: Stacking



In [ ]:
# Create Stacking Ensemble 1
# with Logistic Regression, SVM, Random Forest and XGBOOST
# Meta-model- Logistic Regression

# Define base estimators (same as before)
estimators = [('lr',lr), ('svm', svm), ('rf', rf), ('xgb', xgb_model)]

# Define a meta-model for stacking (you can experiment with different models)
meta_model = LogisticRegression(max_iter=1000, random_state=SEED)

# Create Stacking Classifier ensemble
stacking_ensemble_1 = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_model,
    cv=5  # 5-fold cross-validation for the meta-model
)
stacking_ensemble_1.fit(X_train, y_train)
y_pred_stacking_1 = stacking_ensemble_1.predict(X_test)

print("\nStacking Ensemble 1 Accuracy:", accuracy_score(y_test, y_pred_stacking_1))
print("Classification Report for Stacking Ensemble 1:")
print(classification_report(y_test, y_pred_stacking_1))
print("Confusion Matrix for Stacking Ensemble 1:")
print(confusion_matrix(y_test, y_pred_stacking_1))


# Create Stacking Ensemble 2
# with Logistic Regression, SVC, Random Forest and XGBOOST
# Meta-model- SVC

# Define base estimators (same as before)
estimators = [('lr',lr), ('svm', svm), ('rf', rf), ('xgb', xgb_model)]

# Define a meta-model for stacking (you can experiment with different models)
meta_model = SVC(max_iter=1000, random_state=SEED)

# Create Stacking Classifier ensemble
stacking_ensemble_2 = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_model,
    cv=5  # 5-fold cross-validation for the meta-model
)
stacking_ensemble_2.fit(X_train, y_train)
y_pred_stacking_2 = stacking_ensemble_2.predict(X_test)

print("\nStacking Ensemble 2 Accuracy:", accuracy_score(y_test, y_pred_stacking_2))
print("Classification Report for Stacking Ensemble 2:")
print(classification_report(y_test, y_pred_stacking_2))
print("Confusion Matrix for Stacking Ensemble 2:")
print(confusion_matrix(y_test, y_pred_stacking_2))


# Create Stacking Ensemble 3
# with Logistic Regression, SVM, Random Forest and XGBOOST
# Meta-model- Random Forest Classifier

# Define base estimators (same as before)
estimators = [('lr',lr), ('svm', svm), ('rf', rf), ('xgb', xgb_model)]

# Define a meta-model for stacking (you can experiment with different models)
meta_model = RandomForestClassifier(random_state=42)

# Create Stacking Classifier ensemble
stacking_ensemble_3 = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_model,
    cv=5  # 5-fold cross-validation for the meta-model
)
stacking_ensemble_3.fit(X_train, y_train)
y_pred_stacking_3 = stacking_ensemble_3.predict(X_test)

print("\nStacking Ensemble 3 Accuracy:", accuracy_score(y_test, y_pred_stacking_3))
print("Classification Report for Stacking Ensemble 3:")
print(classification_report(y_test, y_pred_stacking_3))
print("Confusion Matrix for Stacking Ensemble 3:")
print(confusion_matrix(y_test, y_pred_stacking_3))


# Create Stacking Ensemble 4
# with Logistic Regression, Random Forest and XGBOOST
# Meta-model- Logistic Regression

estimators = [('lr', lr), ('rf', rf), ('xgb', xgb_model)]

# Define a meta-model for stacking (you can experiment with different models)
meta_model = LogisticRegression(random_state=SEED, max_iter=1000)

# Create Stacking Classifier ensemble
stacking_ensemble_4 = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_model,
    cv=5  # 5-fold cross-validation for the meta-model
)
stacking_ensemble_4.fit(X_train, y_train)
y_pred_stacking_4 = stacking_ensemble_4.predict(X_test)

print("\nStacking Ensemble 4 Accuracy:", accuracy_score(y_test, y_pred_stacking_4))
print("Classification Report for Stacking Ensemble 4:")
print(classification_report(y_test, y_pred_stacking_4))
print("Confusion Matrix for Stacking Ensemble 4:")
print(confusion_matrix(y_test, y_pred_stacking_4))

##Model Evaluation

In [ ]:
# Create a dictionary of model predictions
model_predictions = {
    "Logistic Regression": y_pred_lr,
    "SVM": y_pred_svm,
    "Random Forest": y_pred_rf,
    "XGBoost": y_pred_xgb,
    "Voting Ensemble": y_pred_voting_3,
    "Stacking Ensemble": y_pred_stacking_3,

}

# Create a list to hold performance results for each model
results = []

# Compute evaluation metrics for each model
for model_name, y_pred in model_predictions.items():
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    results.append({
        "Model": model_name,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1 Score": f1
    })

# Convert results to a DataFrame for a neat tabular view
results_df = pd.DataFrame(results)
print("Performance Comparison Table:")
print(results_df)

# --- Visualization ---
# We can visualize each metric in a separate bar plot.
metrics = ["Accuracy", "Precision", "Recall", "F1 Score"]

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for ax, metric in zip(axes, metrics):
    sns.barplot(x="Model", y=metric, data=results_df, ax=ax)
    ax.set_title(metric)
    ax.set_ylim(0, 1)  # Set limits between 0 and 1
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
    ax.grid(True, linestyle="--", alpha=0.7)

plt.tight_layout()
plt.show()


##Analyze Feature Importance using gain metric

In [ ]:
#analyzing feature importance

xgb.plot_importance(xgb_model, max_num_features=10, importance_type='gain')
plt.title("Feature Importance Using XGBoost")
plt.show()

##Hyperparameter Tuning


In [ ]:
# Search Best Parameters for Baseline Models

# Logistic Regression
lr_params = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
    'solver': ['liblinear', 'lbfgs', 'saga']
}

# SVC
svc_params = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

# Random Forest
rf_params = {
    'n_estimators': [50, 100, 200, 300, 500],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# XGBoost
xgb_params = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}


In [ ]:
# Perform RandomizedSearchCV on each model
lr_search = RandomizedSearchCV(LogisticRegression(random_state=SEED), lr_params, cv=5, n_iter=10, random_state=SEED, n_jobs=-1)
svc_search = RandomizedSearchCV(SVC(probability=True, random_state=SEED), svc_params, cv=5, n_iter=10, random_state=SEED, n_jobs=-1)
rf_search = RandomizedSearchCV(RandomForestClassifier(random_state=SEED), rf_params, cv=5, n_iter=10, random_state=SEED, n_jobs=-1)
xgb_search = RandomizedSearchCV(XGBClassifier(random_state=SEED, eval_metric='mlogloss'), xgb_params, cv=5, n_iter=10, random_state=SEED, n_jobs=-1)

# Fit models
lr_search.fit(X_train, y_train)
svc_search.fit(X_train, y_train)
rf_search.fit(X_train, y_train)
xgb_search.fit(X_train, y_train)

# Get best models
best_lr = lr_search.best_estimator_
best_svc = svc_search.best_estimator_
best_rf = rf_search.best_estimator_
best_xgb = xgb_search.best_estimator_

# Print best parameters
print("Best Logistic Regression:", lr_search.best_params_)
print("Best SVC:", svc_search.best_params_)
print("Best Random Forest:", rf_search.best_params_)
print("Best XGBoost:", xgb_search.best_params_)


In [ ]:
# Generate predictions on the test set using tuned models
# ------------------------------------------------------
y_pred_tuned_lr = best_lr.predict(X_test)
y_pred_tuned_svc = best_svc.predict(X_test)
y_pred_tuned_rf = best_rf.predict(X_test)
y_pred_tuned_xgb = best_xgb.predict(X_test)

# --- Tuned Model Evaluations ---

print("### Tuned Models ###\n")

# Logistic Regression (Tuned)
print("Tuned Logistic Regression:")
print("Accuracy:", accuracy_score(y_test, y_pred_tuned_lr))
print("Classification Report:")
print(classification_report(y_test, y_pred_tuned_lr))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_tuned_lr))
print("\n------------------------\n")

# Support Vector Classifier (Tuned)
print("Tuned Support Vector Classifier:")
print("Accuracy:", accuracy_score(y_test, y_pred_tuned_svc))
print("Classification Report:")
print(classification_report(y_test, y_pred_tuned_svc))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_tuned_svc))
print("\n------------------------\n")

# Random Forest (Tuned)
print("Tuned Random Forest:")
print("Accuracy:", accuracy_score(y_test, y_pred_tuned_rf))
print("Classification Report:")
print(classification_report(y_test, y_pred_tuned_rf))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_tuned_rf))
print("\n------------------------\n")

# XGBoost (Tuned)
print("Tuned XGBoost:")
print("Accuracy:", accuracy_score(y_test, y_pred_tuned_xgb))
print("Classification Report:")
print(classification_report(y_test, y_pred_tuned_xgb))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_tuned_xgb))

In [ ]:
# Hyperparameter Tuning
# --- Voting Ensemble with tuned base models ---
voting_ensemble_1 = VotingClassifier(
    estimators=[('lr', best_lr), ('rf', best_rf), ('xgb', best_xgb)],
    voting='hard'  # or 'soft'
)
voting_ensemble_1.fit(X_train, y_train)

# Predict using the tuned Voting1 Ensemble
y_pred_tuned_voting_1 = voting_ensemble_1.predict(X_test)
print("Voting Ensemble (tuned) Accuracy:", accuracy_score(y_test, y_pred_tuned_voting_1))
print(classification_report(y_test, y_pred_tuned_voting_1))


voting_ensemble_2 = VotingClassifier(
    estimators=[('lr', best_lr), ('svm', best_svc), ('rf', best_rf),('xgb', best_xgb)],
    voting='hard'  # or use 'soft' if models support predict_proba()
)
voting_ensemble_2.fit(X_train, y_train)
y_pred_voting_2 = voting_ensemble_2.predict(X_test)

print("Voting Ensemble 2 Accuracy:", accuracy_score(y_test, y_pred_voting_2))
print("Classification Report for Voting Ensemble 2:")
print(classification_report(y_test, y_pred_voting_2))


voting_ensemble_3 = VotingClassifier(
    estimators=[('svm', best_svc), ('rf', best_rf),('xgb', best_xgb)],
    voting='hard'  # or use 'soft' if models support predict_proba()
)
voting_ensemble_3.fit(X_train, y_train)
y_pred_voting_3 = voting_ensemble_3.predict(X_test)

print("Voting Ensemble 3 Accuracy:", accuracy_score(y_test, y_pred_voting_3))
print("Classification Report for Voting Ensemble 3:")
print(classification_report(y_test, y_pred_voting_3))


In [ ]:
#Hyperparameter Tuning
# --- Stacking Ensemble with tuned base models ---

# Tuned Stacking Ensemble 1
stacking_ensemble_1= StackingClassifier(
    estimators=[('lr', best_lr), ('rf', best_rf), ('xgb', best_xgb)],
    final_estimator=LogisticRegression(random_state=SEED, max_iter=1000)
)
stacking_ensemble_1.fit(X_train, y_train)

# Predict using the tuned Stacking Ensemble
y_pred_tuned_stack_1 = stacking_ensemble_1.predict(X_test)
print("Stacking Ensemble (tuned) Accuracy:", accuracy_score(y_test, y_pred_tuned_stack_1))
print(classification_report(y_test, y_pred_tuned_stack_1))


# Tuned Stacking Ensemble 2
stacking_ensemble_2= StackingClassifier(
    estimators=[('lr', best_lr), ('rf', best_rf), ('xgb', best_xgb)],
    final_estimator=SVC(random_state=SEED, max_iter=1000)
)
stacking_ensemble_2.fit(X_train, y_train)

# Predict using the tuned Stacking Ensemble
y_pred_tuned_stack_2 = stacking_ensemble_2.predict(X_test)
print("Stacking Ensemble (tuned) Accuracy:", accuracy_score(y_test, y_pred_tuned_stack_2))
print(classification_report(y_test, y_pred_tuned_stack_2))


# Tuned Stacking Ensemble 3
stacking_ensemble_3= StackingClassifier(
    estimators=[('lr', best_lr), ('rf', best_rf), ('xgb', best_xgb)],
    final_estimator=RandomForestClassifier(random_state=SEED)
)

stacking_ensemble_3.fit(X_train, y_train)

# Predict using the tuned Stacking Ensemble
y_pred_tuned_stack_3 = stacking_ensemble_3.predict(X_test)
print("Stacking Ensemble (tuned) Accuracy:", accuracy_score(y_test, y_pred_tuned_stack_3))
print(classification_report(y_test, y_pred_tuned_stack_3))


## Model Performance

In [ ]:
# Evaluate model performance with AUC/ROC

# List of models
models = {
    "Logistic Regression": best_lr,
    "SVC": best_svc,
    "Random Forest": best_rf,
    "XGBoost": best_xgb,
    "Voting Ensemble": voting_ensemble_1,
    "Stacking Ensemble": stacking_ensemble_1
}

In [ ]:
# Plotting the Curve

plt.figure(figsize=(10, 6))

for model_name, model in models.items():
    # Check if the model has predict_proba
    if hasattr(model, 'predict_proba'):
        # Get probability scores
        y_prob = model.predict_proba(X_test)[:, 1]  # Select probabilities for positive class

        # Compute ROC curve
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)

        # Plot ROC curve
        plt.plot(fpr, tpr, label=f"{model_name} (AUC = {roc_auc:.2f})")
    else:
        print(f"Skipping ROC curve for {model_name} as it does not have predict_proba.")

# Plot random guess line
plt.plot([0, 1], [0, 1], linestyle="--", color="gray", label="Random Guess")

# Labels & Legend
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve Comparison")
plt.legend()
plt.show()

# Final Model Evaluation

In [ ]:
# Create a dictionary mapping model names to their predictions
# ------------------------------
model_predictions = {
    "LR (baseline)": y_pred_lr,
    "SVM (baseline)": y_pred_svm,
    "RF (baseline)": y_pred_rf,
    "XGB (baseline)": y_pred_xgb,
    "LR (tuned)": y_pred_tuned_lr,
    "SVM (tuned)": y_pred_tuned_svc,
    "RF (tuned)": y_pred_tuned_rf,
    "XGB (tuned)": y_pred_tuned_xgb,
    "Voting (ensemble)": y_pred_voting_3,
    "Stacking (ensemble)": y_pred_stacking_3,
    "Voting (tuned ensemble)": y_pred_tuned_voting_1,
    "Stacking (tuned ensemble)": y_pred_tuned_stack_1
}

# ------------------------------
# Compute evaluation metrics for each model
# ------------------------------
results = []
for model_name, y_pred in model_predictions.items():
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    results.append({
        "Model": model_name,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1 Score": f1
    })

# Convert results to a DataFrame for a neat tabular view
results_df = pd.DataFrame(results)
print("Performance Comparison Table:")
print(results_df)

# ------------------------------
# Visualization of Metrics
# ------------------------------
# We'll create a 2x2 grid for the four metrics.
metrics = ["Accuracy", "Precision", "Recall", "F1 Score"]
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for ax, metric in zip(axes, metrics):
    sns.barplot(x="Model", y=metric, data=results_df, ax=ax, hue="Model", palette="viridis", dodge=False, legend=False) # Changed here
    ax.set_title(metric, fontsize=14)
    ax.set_ylim(0, 1)
    ax.set_xlabel("")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right", fontsize=12)
    ax.grid(True, linestyle="--", alpha=0.7)

plt.tight_layout()
plt.show()